$$
\newcommand{\mat}[1]{\boldsymbol {#1}}
\newcommand{\mattr}[1]{\boldsymbol {#1}^\top}
\newcommand{\matinv}[1]{\boldsymbol {#1}^{-1}}
\newcommand{\vec}[1]{\boldsymbol {#1}}
\newcommand{\vectr}[1]{\boldsymbol {#1}^\top}
\newcommand{\rvar}[1]{\mathrm {#1}}
\newcommand{\rvec}[1]{\boldsymbol{\mathrm{#1}}}
\newcommand{\diag}{\mathop{\mathrm {diag}}}
\newcommand{\set}[1]{\mathbb {#1}}
\newcommand{\cset}[1]{\mathcal{#1}}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
\newcommand{\pderiv}[2]{\frac{\partial #1}{\partial #2}}
\newcommand{\bb}[1]{\boldsymbol{#1}}
\newcommand{\E}[2][]{\mathbb{E}_{#1}\left[#2\right]}
\newcommand{\ip}[3]{\left<#1,#2\right>_{#3}}
\newcommand{\given}[]{\,\middle\vert\,}
\newcommand{\DKL}[2]{\cset{D}_{\text{KL}}\left(#1\,\Vert\, #2\right)}
\newcommand{\grad}[]{\nabla}
\newcommand{\norm}[1]{\left\lVert#1\right\rVert}
$$

# Part 4: Summary Questions
<a id=part4></a>

This section contains summary questions about various topics from the course material.

You can add your answers in new cells below the questions.

**Notes**

- Clearly mark where your answer begins, e.g. write "**Answer:**" in the beginning of your cell.
- Provide a full explanation, even if the question doesn't explicitly state so. We will reduce points for partial explanations!
- This notebook should be runnable from start to end without any errors.

### CNNs

1. Explain the meaning of the term "receptive field" in the context of CNNs.
**Answer:**
The recepetive field of a neuron is a measure of how many other neurons in earlier layers affect it.
We can extend this definition and generalize how a specific feature is affected by a portion of space in earlier layers.
The recpetive field helps us understand certain feature maps in the network are affected, and to get a notion of what is the extent of the input data within the architecture.
The recpetive field is affected b filter size, pooling layers, and certain paramers such as dialation, stride etc.


2. Explain and elaborate about three different ways to control the rate at which the receptive field grows from layer to layer. Compare them to each other in terms of how they combine input features.

**Answer:** Three ways to manipulate the receptive field are as follows:

**Kernel Size** : The kernel size explicitly determines how many neurons participate in the calculation of a single neuron in the following layer. Therefore, increasing the kernel size will increase the receptive of a neuron.
The kernel size maintains information from adjacent cells and defines to what extent is the adjacency between cells.

**Stride** : The stride determines the spacing between each consecutive convolution (correlation) operation in adjacent cells, and therefore increasing the stride will effectively increase the receptive field, this takes effect for 2 consecutive layers or more, as in the context of only one layer, the receptive field will remain the same.
The stride combines adjacent cells information and decreases output size (proportional to the stride factor).

**Dilation** : Effectively expands the kernel size by padding zeros between the kernel cells, and therefore the receptive field increases as dilation increases.
The dilation combines input features that are evenly spaced (by the dilation factor) into a single neuron.

Stride and dilation increase the receptive field, however they do not increase the actual number of neurons which were taken into accounts, and only in the manner the spatial information (and support of earlier layers).
In contrast increasing the kernel size does not only increase the spatial information, but also adds more neurons that are considered.


3. Imagine a CNN with three convolutional layers, defined as follows:

In [1]:
import torch
import torch.nn as nn

cnn = nn.Sequential(
    nn.Conv2d(in_channels=3, out_channels=4, kernel_size=3, padding=1),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=4, out_channels=16, kernel_size=5, stride=2, padding=2),
    nn.ReLU(),
    nn.MaxPool2d(2),
    nn.Conv2d(in_channels=16, out_channels=32, kernel_size=7, dilation=2, padding=3),
    nn.ReLU(),
)

cnn(torch.rand(size=(1, 3, 1024, 1024), dtype=torch.float32)).shape

torch.Size([1, 32, 122, 122])

What is the size (spatial extent) of the receptive field of each "pixel" in the output tensor?

**Answer:** 
Padding output does changes the output size, however does not affect the recptive field.
When using dialation, we can consider an effective kernel size of $k'=d(k-1)$, where $d$ is the dilation and $k$ is the kernel size.
Looking at a single convolution layer in 1D, we can take the current receptive field of the neuron, and look at the 2 neurons at the edge and calculate how they contribute the the recepetive field of the next layer.
Each edge neuron is affected by the effective kernel size, half of it and the center neron is already in the older recpetive field, so we end with expanding by the effective kernel size minus one overall accounting for two edge neourons.


We can come up with the following formula:

$r_{i}=r_{i-1}+d(k-1)$

Using stride, we are actually mutiplying the receptive of the neuron by the stride factor from that layer and onwards. We can incoporate this into the formula with an additional parameter $n$:

$n_{i}=n_{i-1}\cdot s$

And come up with the following:

$r_{i}=r_{i-1}+d(k-1)\cdot n_{0}\ \ n_{i}=n_{i-1}\cdot s$

$r_{i}-\text{receptive field of a neuron in layer i},\ d-\text{dilation},\ k-\text{kernel size},\ s-\text{stride}$


We can treat pooling layers as a conv-layer with a fixed value for each cell ($=\frac{1}{K^{2}}$) and a stride of 2.


This gives us: 

$r_{0}=1$

$\text{Conv 1:}r_{1}=1+1(3-1)=3$

$\text{Pool 1:}r_{2}=3+1(2-1)\cdot2=5$

$\text{Conv 2:}r_{3}=5+1(5-1)\cdot2\cdot2=21$

$\text{Pool 2:}r_{4}=21+1(2-1)\cdot2\cdot2\cdot2=29$

$\text{Conv 3:}r_{5}=29+2(7-1)\cdot2\cdot2\cdot2=125$

As this is performed as a 2D conv, we can infer that the recpetive field of a neuron at the out is $\boxed {125 \times 125}$

4. You have trained a CNN, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$, and $f_l(\cdot;\vec{\theta}_l)$ is a convolutional layer (not including the activation function).

  After hearing that residual networks can be made much deeper, you decide to change each layer in your network you used the following residual mapping instead $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)+\vec{x}$, and re-train.

  However, to your surprise, by visualizing the learned filters $\vec{\theta}_l$ you observe that the original network and the residual network produce completely different filters. Explain the reason for this.
  
**Answer:** When training a residual block, the block learns a residual mapping which is a delta from the identity mapping as we learned in the tutorial, and is much easier to opimize.
On contrast when training a "classic" CNN block, the filter is learnt "from scratch" so we would expect a major difference between the filter of the CNN (which is learnt "from scratch") and of the residual block (Which is learnt from the identity mapping).

### Dropout

1. **True or false**: dropout must be placed only after the activation function.

**Answer:** **False**. 
Assuming the activation performed maps $0$ to $0$, then performing a dropout after the activation has the same results, since we would zero certain neurons before or after the activation, and the result will be the same.

It might seem more sensical to place dropout before activation in order to be more efficient (do not perform calculation which we would zero anyway).

2. After applying dropout with a drop-probability of $p$, the activations are scaled by $1/(1-p)$. Prove that this scaling is required in order to maintain the value of each activation unchanged in expectation.

Lets us denote the expectation value before and after the drop out as follows:

$x-\text{layer without dropout },E[\varphi(x)]=\alpha$

$y-\text{layer with dropout },y=\begin{cases}
x & 1-p\\
0 & p
\end{cases}$

$E[\varphi(y)]=E[\mathbb{I}\left\{ 1-p\right\} \varphi(x)]=E[\mathbb{I}\left\{ 1-p\right\} ]E[\varphi(x)]=(1-p)\alpha$

Where $\mathbb{I}$ is an indicator function, and as we know the expectation of an indicator is the propability of the event. We also assume that the layer and the dropout probability are statistically independent.
Scaling by the said factor will result in the same expectation:
$\hat{\varphi}(x)-\text{layer with dropout scaling},\hat{\varphi}(x)=\begin{cases}
\frac{1}{1-p}\cdot\varphi(x) & 1-p\\
0 & p
\end{cases}$

$E[\hat{\varphi}(x)]=E[\mathbb{I}\left\{ 1-p\right\} \frac{1}{1-p}\varphi(x)]=E[\mathbb{I}\left\{ 1-p\right\} ]\frac{1}{1-p}E[\varphi(x)]=\frac{1-p}{1-p}\alpha=E[\varphi(x)]$

### Losses and Activation functions

1. You're training a an image classifier that, given an image, needs to classify it as either a dog (output 0) or a hotdog (output 1). Would you train this model with an L2 loss? if so, why? if not, demonstrate with a numerical example. What would you use instead?

**Answer:** As learnt, when dealinig with a classification problem, and specifically the binary classification problem, it will be preferable to use a binary cross-entropy loss.
If we will use a L2 loss, we will create a loss function which is also binary, since we will always calculate the difference between ones and zeros.
This results in a non-continous function (and non derivatable), which will be problematic when training with gradient descent methods, as gradients will be approximately zero, and the optimization won't be effective.

2. After months of research into the origins of climate change, you observe the following result:

<center><img src="https://sparrowism.soc.srcf.net/home/piratesarecool4.gif" /></center>

You decide to train a cutting-edge deep neural network regression model, that will predict the global temperature based on the population of pirates in `N` locations around the globe.
You define your model as follows:

In [2]:
import torch.nn as nn

N = 42  # number of known global pirate hot spots
H = 128
mlpirate = nn.Sequential(
    nn.Linear(in_features=N, out_features=H),
    nn.Sigmoid(),
    *[
        nn.Linear(in_features=H, out_features=H),
        nn.Sigmoid(),
    ]*N,
    nn.Linear(in_features=H, out_features=1),
)

While training your model you notice that the loss reaches a plateau after only a few iterations.
It seems that your model is no longer training.
What is the most likely cause?

**Answer:** 

It seems that the training process encountered a vanishing gradient problem.
As this network is quite deep, and the sigmoid funciton maps the entire input space into $(0,1)$, we would expect that even large changes that pass through the activation would result in small gradients (the sigmoid is quite a plateau with large negative and positive values).
Because we chain many sigmoid activation consecutively, the gradients become even smaller, and vanish. Effectively we won't train the network, and we wouldn't notice any changes as seen in this scenario.

3. Referring to question 2 above: A friend suggests that if you replace the `sigmoid` activations with `tanh`, it will solve your problem. Is he correct? Explain why or why not.

**Answer:** The $tanh$ suffers from the same problem as it also maps the entire input space into a relatively small region of $(-1,1)$. The derivative of $tanh$ tends to be also low with large values, and when using deep network we would expect a similar behaviour as with the sigmoid.

4. Regarding the ReLU activation, state whether the following sentences are **true or false** and explain:
    
    A. In a model using exclusively ReLU activations, there can be no vanishing gradients.
    
      **Answer**: **true**. Relu's derivative is a constant 1 or 0, so there is no notion of a gradient diminishing as when chaining multiple ReLUs still result in a binary value.
      
    B. The gradient of ReLU is linear with its input when the input is positive.
    
    **Answer**: **false**. If Relu's **value** is linear when the input is positive, however the gradient will be a constant 1, which is not linear with the input.
    
    C. ReLU can cause "dead" neurons, i.e. activations that remain at a constant value of zero.
    
    **Answer**: **true** once ReLU is activated on a negative value, the gradient will result in zero. chaining this zero to other gradient will still remain zero, thus creating a "dead neuron" which does not affect the step.
    

### Optimization

1. Explain the difference between: stochastic gradient descent (SGD), mini-batch SGD and regular gradient descent (GD).

**Answer**

GD performs an optimization step based on the whole training set (broken into batches) before performing an actual step. So it is only one step of gradient descent in one epoch.

On the other side of the spectrum SGD performs each step based on a single sample. so and epich is broken into an amount of steps equivalent to the amoount of samples.

In mini-batch SGD we use a batch with a fixed number of samples which is called a mini-batch. We then optimize a step based on a single mini batch. an epoch ends when we've cycled through all mini batches.

2. Regarding SGD and GD:
    1. Provide at least two reasons for why SGD is used more often in practice compared to GD.
    **Answer**
    When training large dataset, it might be unfeasable to calculate each and every gradient of our training set per step (resource-wise), therefore breaking the epoch into mini-batches helps this process of computaiton.
    
    Moreover as we learnt in previous HW, when optimizing with GD we might get stuck in a local minima. The fact that SGD is calculated from 1 sample would probably result in different gradient curves which might pull us from a local minima.
    
    2. In what cases can GD not be used at all?
    
    **Answer**
    When there is no sufficient resources (memory) for storing all inputs and gradient data in the memory, we wouldn't be able to use the GD as it uses the entire training set.

3. You have trained a deep resnet to obtain SoTA results on ImageNet.
While training using mini-batch SGD with a batch size of $B$, you noticed that your model converged to a loss value of $l_0$ within $n$ iterations (batches across all epochs) on average.
Thanks to your amazing results, you secure funding for a new high-powered server with GPUs containing twice the amount of RAM.
You're now considering to increase the mini-batch size from $B$ to $2B$.
Would you expect the number of of iterations required to converge to $l_0$ to decrease or increase when using the new batch size? explain in detail.

**Answer**: 

Let us neglect for a moment the problem of local-minimas that may arise when using a larger mini-batch size.
We have learnt about a trade-off in the accuracy and speed of convergence when deciding a mini-batch size.
When increasing the mini-batch size to $2B$, we better approximate the gradient toward the minima as the model proccesses the same data in less iterations.
Meaning we will be able to reach $l_0$ in less iterations.

However we talked in previous sections regarding introducing "noise" into the GD method that will enable us to evade getting stuck in a local minima. Increasing the mini-batch size introduces greater risk of that problem to rise again.

In that case we would say it would take more iteration to reach $l_0$ as the model needs to work harder to evade the local minima (if possible).

4. For each of the following statements, state whether they're **true or false** and explain why.
    1. When training a neural network with SGD, every epoch we perform an optimization step for each sample in our dataset.
    1. Gradients obtained with SGD have less variance and lead to quicker convergence compared to GD.
    1. SGD is less likely to get stuck in local minima, compared to GD.
    1. Training  with SGD requires more memory than with GD.
    1. Assuming appropriate learning rates, SGD is guaranteed to converge to a local minimum, while GD is guaranteed to converge to the global minimum.
    1. Given a loss surface with a narrow ravine (high curvature in one direction): SGD with momentum will converge more quickly than Newton's method which doesn't have momentum.
    
    **Answer**: 
    1. **True**. As we explained in previous sections, we define an epoch to be an iteration over all our dataset. As the SGD breaks the dataset into single sample on which we perform a step, the epoch will end after performing a step on each sample.
    1. **False**. Although we can argue SGD may converge faster than GD in some situations, The variance of SGD will be higher since every step is performed on a single sample which may be completely different from the average of all samples (as in GD) or a group of samples (as in mini-batch GD). On the hand GD average the samples into a single gradient directions, therefore we can expect a lower variance.
    1. **True**. As we discussed in previous answers, SGD introduces "noise" to the optimization, and the fact that our gradient is changing every step, we are able this way to evade a local minima which in GD will be present in each step(=epoch).
    1. **False**. SGD breaks the dataset into single samples each step, therefore we need only to store that single sample and its gradient in the model. Therefore it would require less memory from GD which would requires storing the entire training set and gradients when training.
    1. **False**. Assuming appropriate learning rate, and as we speak of non-convex problems, both SGD and GD is guaranteed to converge into a local-minima (which may be sometimes the global one). However as we said it is more likely that SGD will coverge faster and less probable that it will get stuck in a local minima which GD suffered (and had a higher loss)
    1. **False**. If an SGD has a too much momentum, it might skip the minima which in newton's method would be achieved (Newton's method does not use momentum)

5. In tutorial 5 we saw an example of bi-level optimization in the context of deep learning, by embedding an optimization problem as a layer in the network.
    1. **True or false**: In order to train such a network, the inner optimization problem must be solved with a descent based method (such as SGD, LBFGS, etc).
  Provide a mathematical justification for your answer.
  
   **Answer**: **False**. We can denote the inner-optimization problem as follows:
   
   
   $\Theta^{*}=\arg\min_{\Theta}\left\{ \mathbb{E}_{(x,y)\sim D}\left[\mathcal{L}\left(y,\arg\min_{y\prime}f(y\prime,h_{\Theta}(x)\right)\right]\right\} $
   
   As we learned, looking at the inner problem, $z=h_{\Theta}(x)$ is contant, therefore we can reduce the scope of solving it only in the parameter $y\prime$.
   
   Solving $y=argmin_{y′}f(y′,z)$ requires calculating hessians of $f$ w.r.t $y,z$. After calculating hessians the problem concludes into solving (for the inner problem):
   $∂y/∂z=−R^⊤K^{−1}.$
   
   This problem can be solved in several ways and is not limited to GD methods. E.G we can model it as a LLS problem and have a closed solution using a Moore-Penrose inverse (pseudo-inverse).
  
   
   

6. You have trained a neural network, where each layer $l$ is represented by the mapping $\vec{y}_l=f_l(\vec{x};\vec{\theta}_l)$ for some arbitrary parametrized functions $f_l(\cdot;\vec{\theta}_l)$.
  Unfortunately while trying to break the record for the world's deepest network, you discover that you are unable to train your network with more than $L$ layers.
    1. Explain the concepts of "vanishing gradients", and "exploding gradients".
    2. How can each of these problems be caused by increased depth?
    3. Provide a numerical example demonstrating each.
    4. Assuming your problem is either of these, how can you tell which of them it is without looking at the gradient tensor(s)?
    
    **Answer**: 
    
    1. Vanishing gradients occurs in GD methods when gradients often get smaller and approach zero as they are chained through the layers in the network (leveraging the chain rule). This causes the weights of lower layers remain nearly unchanged. This way we do not converge into an optimum.
    Exploding gradients on the other hand depicts the problem of gradients getting larger as they backpropagate through the network. This will cause large weight updates in layers, and in turn will cause the GD to diverge.

    2. Since we leverage the chain-rule when calculating weight updates, we are always multiplying gradients from one layer to another. Therfore if we have really low gradients - then their product will be even lower. If gradients have a large value, then their product will be even higher (exponentially).
    
    3. Say we have a constant gradient value of 0.5 in each layer, and we have 10 layers. in th 10th layer the update rule will have a value of 0.5, however in the first layer we will have a value of $\left(\frac{1}{2}\right)^{10}$ which gets lower as we increase layers.
    Similarly, let us have a constant gradient value of 2 with 10 layer. The 10th layer will update with a value of 2, and the first layer with a value of $\left(2\right)^{10}$, and this gets higher as the networks deepends.
       
    4. We can inspect the loss graph in the training - if we experience an platea behaviiur which converges quite fast in iterations (and has a low performance) we can assume it is the vanishing gradients.
    If we notice a divergence behaviour and fluctuating vales of loss, we can assume this is the exploding gradients problem.

### Backpropagation

1. You wish to train the following 2-layer MLP for a binary classification task:
  $$
  \hat{y}^{(i)} =\mat{W}_2~ \varphi(\mat{W}_1 \vec{x}^{(i)}+ \vec{b}_1) + \vec{b}_2
  $$
  Your wish to minimize the in-sample loss function is defined as
  $$
  L_{\mathcal{S}} = \frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)}) + \frac{\lambda}{2}\left(\norm{\mat{W}_1}_F^2 + \norm{\mat{W}_2}_F^2 \right)
  $$
  Where the pointwise loss is binary cross-entropy:
  $$
  \ell(y, \hat{y}) =  - y \log(\hat{y}) - (1-y) \log(1-\hat{y})
  $$
  
  Write an analytic expression for the derivative of the final loss $L_{\mathcal{S}}$ w.r.t. each of the following tensors: $\mat{W}_1$, $\mat{W}_2$, $\mat{b}_1$, $\mat{b}_2$, $\mat{x}$.
  

**Answer**:
Leveraging the chain rule: (We assume x is a vector, W's are matrices and b's are vectors)
  
$ \hat{y}^{(i)}=W_{2}\varphi(W_{1}x^{(i)}+b_{1})+b_{2}$

$\ell(y^{(i)},\hat{y}^{(i)})=-y\log\hat{y}-(1-y)\log(1-\hat{y})$

$L_{s}=\frac{1}{N}\sum_{i=1}^{N}\ell(y^{(i)},\hat{y}^{(i)})+\frac{\lambda}{2}\left(\left|\left|W_{1}\right|\right|_{F}^{2}+\left|\left|W_{2}\right|\right|_{F}^{2}\right)$

$\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial\hat{y}^{(i)}}=\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}$

$\frac{\partial L_{s}}{\partial b_{2}}=\frac{1}{N}\sum_{i=1}^{N}\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial b_{2}}=\frac{1}{N}\sum_{i=1}^{N}\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial b_{2}}=\boxed{\frac{1}{N}\sum_{i=1}^{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)}$

$\frac{\partial L_{s}}{\partial W_{2}}=\frac{1}{N}\sum_{i=1}^{N}\left(\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial W_{2}}\right)+\frac{\lambda}{2}\cdot\frac{\partial(\left|\left|W_{2}\right|\right|_{F}^{2})}{W_{2}}=\boxed{\frac{1}{N}\sum_{i=1}^{N}\left(\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)\cdot\varphi(W_{1}x^{(i)}+b_{1})^{T}\right)+\lambda W_{2}}$

$\frac{\partial L_{s}}{\partial b_{1}}=\frac{1}{N}\sum_{i=1}^{N}\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial b_{1}}=\frac{1}{N}\sum_{i=1}^{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\frac{\partial\varphi(z)}{\partial z}|_{z=W_{1}x^{(i)}+b_{1}}\frac{\partial(W_{1}x^{(i)}+b_{1})}{\partial b_{1}}=\boxed{\frac{1}{N}\sum_{i=1}^{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\varphi'(W_{1}x^{(i)}+b_{1})}$

$\frac{\partial L_{s}}{\partial W_{1}}=\frac{1}{N}\sum_{i=1}^{N}\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial W_{1}}+\frac{\lambda}{2}\cdot\frac{\partial(\left|\left|W_{1}\right|\right|_{F}^{2})}{W_{1}}=\frac{1}{N}\sum_{i=1}^{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\frac{\partial\varphi(z)}{\partial z}|_{z=W_{1}x^{(i)}+b_{1}}\frac{\partial(W_{1}x^{(i)}+b_{1})}{\partial W_{1}}+\lambda W_{1}$

$=\boxed{\frac{1}{N}\sum_{i=1}^{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\varphi'(W_{1}x^{(i)}+b_{1})\left(x^{(i)}\right)^{T}+\lambda W_{1}}$



$\frac{\partial L_{s}}{\partial x^{(j)}}=\frac{1}{N}\sum_{i=1}^{N}\left(\frac{\partial\ell(y^{(i)},\hat{y}^{(i)})}{\partial\hat{y}^{(i)}}\cdot\frac{\partial\hat{y}^{(i)}}{\partial x^{(j)}}\right)=\frac{1}{N}\sum_{i=1}^{N}\left(\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\frac{\partial\varphi(z)}{\partial z}|_{z=W_{1}x^{(i)}+b_{1}}\frac{\partial(W_{1}x^{(i)}+b_{1})}{\partial x^{(j)}}\right)\underset{\text{non zero for {i=j}}}{=}\frac{1}{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\varphi'(W_{1}x^{(j)}+b_{1})W_{1}^{T}$

And in a vector fashion:

$\frac{\partial L_{s}}{\partial x}=\boxed{\frac{1}{N}\left(\frac{1-y}{1-\hat{y}}-\frac{y}{\hat{y}}\right)W_{2}^{T}\varphi'(W_{1}x+b_{1})W_{1}^{T}}$

2. Given the following code snippet, implement the custom backward function `part4_affine_backward` in `hw4/answers.py` so that it passes the `assert`s.

In [26]:
from torch.autograd import Function

from hw4.answers import part4_affine_backward

N, d_in, d_out = 100, 11, 7
dtype = torch.float64
X = torch.rand(N, d_in, dtype=dtype)
W = torch.rand(d_out, d_in, requires_grad=True, dtype=dtype)
b = torch.rand(d_out, requires_grad=True, dtype=dtype)

def affine(X, W, b):
    return 0.5 * X @ W.T + b

class AffineLayerFunction(Function):
    @staticmethod
    def forward(ctx, X, W, b):
        result = affine(X, W, b)
        ctx.save_for_backward(X, W, b)
        return result

    @staticmethod
    def backward(ctx, grad_output):
        return part4_affine_backward(ctx, grad_output)

l1 = torch.sum(AffineLayerFunction.apply(X, W, b))
l1.backward()
W_grad1 = W.grad
b_grad1 = b.grad

l2 = torch.sum(affine(X, W, b))
W.grad = b.grad = None
l2.backward()
W_grad2 = W.grad
b_grad2 = b.grad

assert torch.allclose(W_grad1, W_grad2)
assert torch.allclose(b_grad1, b_grad2)

### Sequence models

1. Regarding word embeddings:
    1. Explain this term and why it's used in the context of a language model.
    1. Can a language model like the sentiment analysis example from the tutorials be trained without an embedding (i.e. trained directly on sequences of tokens)? If yes, what would be the consequence for the trained model? if no, why not?
    
    **Answer**
    
    1. word embedding is a representation of a literal word in a numerical vector with a dimension. The word embeddings are in represented in the embedding space such that each literal word has an ebdding vector closest to its meaning in the embedding space.
    Words with close semantic meaning will have close vector representations in the embeddings space (in a norm fashion)
    
    We use this modeling because:
    
    - The langauage requires a numerical representation in our model, and so word embedding is one of the methods used for representing it.
    - The embedding space is quite efficient in terms of scalability and sparsity, since representing words in a vocabulary in an $N$ dimensional embedding vector populates all cells rather than using a representation such as one-hot vector which scales linearly with the number samples which can come up to millions or more bits, and does not populate all cells and thus is very sparse and not efficient for calculations.
    - Moreover we introduce context in our model since we represent an embedding space where distance between vectors may be proportional to semantic link between words.
    
    2. It is possible to train without a word embedding (such as with one-hot encodings) however we assume training such a model will require much more data and time to train (compared to embeddings) since a model like this we try to learn words based on the similarity of them to other learnt words rather than inferring them from the embedding space which represent semantic relations. This semantic problem also persists when we face new data which we didn't train on.

2. Considering the following snippet, explain:
    1. What does `Y` contain? why this output shape?
    2. How you would implement `nn.Embedding` yourself using only torch tensors. 
    
**Answer**

1. Looking at X initialization, we can infer that or input range are numbers between $[0,41]$, and so we should create 42 different embeddings for these values. We can see that for each value we would encode a vector of 42000 values (as seen in embedding_dim).
All in all we would expect Y to be with the same amount of values of X multplied by the embedding dimension.
Therefore we get a tensor of $dim(X)\times42000$.
Y is the embeddings vector (tensor in our case) of X.

2. We would need to create a lookup table of 42 tensors, each one is the size of 42000. let us denote $L\in\mathbb{R}^{42}$. Now each value in $[0,41]$ will represent an index in $L$ which contains its embedding vector.
In that way we can take our input and index our lookup table in a tensor fashion (via stacking values from the tensor and then reshaping them back to the orig shpe) and get the equivalent embedding vector.

In [28]:
import torch.nn as nn

X = torch.randint(low=0, high=42, size=(5, 6, 7, 8))
embedding = nn.Embedding(num_embeddings=42, embedding_dim=42000)
Y = embedding(X)
print(f"{Y.shape=}")

Y.shape=torch.Size([5, 6, 7, 8, 42000])


3. Regarding truncated backpropagation through time (TBPTT) with a sequence length of $S$: State whether the following sentences are **true or false**, and explain.
    1. TBPTT uses a modified version of the backpropagation algorithm.
    2. To implement TBPTT we only need to limit the length of the sequence provided to the model to length $S$.
    3. TBPTT allows the model to learn relations between input that are at most $S$ timesteps apart.
    
    **Answer**
    1. **True** - In TBPTT the we try to limit the vanishing and exploding gradient problems by limiting the number of timesteps to we backpropagate on which is the truncation. afterwards we are continuing the BPTT modeling as normal which utilizes the backpropagation algorithm.
    
    2. **False** as explained previously, we limit the number of timestamps while we propagate and not the length of the sequence.
    
    3. **True** as explained previously, we limit the number of timestamps, which effectively means that we detach the computational graph after $S$ timestamps. Therefore computation reaches only S timestamps apart.

### Attention

1. In tutorial 7 (part 2) we learned how to use attention to perform alignment between a source and target sequence in machine translation.
    1. Explain qualitatively what the addition of the attention mechanism between the encoder and decoder does to the hidden states that the encoder and decoder each learn to generate (for their language). How are these hidden states different from the model without attention?
  
  2. After learning that self-attention is gaining popularity thanks to the shiny new transformer models, you decide to change the model from the tutorial: instead of the queries being equal to the decoder hidden states, you use self-attention, so that the keys, queries and values are all equal to the encoder's hidden states (with learned projections). What influence do you expect this will have on the learned hidden states?


**Answer**

A. When not using attention, our decoder in the model will try and interpret context from the entire message from the last hiddent state of the encoder.
Using attention serves as a weighted version of this interpretation since we can give different weights to certain parts of the encoder's message in a particlar decoder state. These weights are also learned as a propability distribution over a sequence of elements, and enables the decoder to "pay more attention" to some elements from the sequence (when they have a higher weight).

B.  Using self attention means using the hidden state from the encoder as a query will introduce more context to the hidden state of the decoder **within the same sequence**. This enables the model to generalize better even when it is trainied only  on a few examples.

### Unsupervised learning

1. As we have seen, a variational autoencoder's loss is comprised of a reconstruction term and  a KL-divergence term. While training your VAE, you accidentally forgot to include the KL-divergence term.
What would be the qualitative effect of this on:

    1. Images reconstructed by the model during training ($x\to z \to x'$)?
    1. Images generated by the model ($z \to x'$)?
    
   
**Answer**
    
A. Images reconstructed by the model during training will not be affected since nullfying the KL-divergence term gives us a basic compressor-decompressor of images, and the reconstruction process is not optimizeed by the KLDIV term.
    We would expect even that the reconstruction of an images might improve, since the optimization problem will not have the KLDIV to regularize, and therefore we would try to reconstruct data as good as possible.
    
B. Images generate by the model will be heavily affected by this change. The KLDIV term enforces the latent space distribution $z$ to have a normal-distribution characteristics, so that when sampling from a normal distibution we would approximate better the sampling from the latent space.
Therefore nullifying the KLDIV term would not enforce any constraints on $z$ and so we would sample a misrepresentation distribution of the latent space (and would prbably generate very non-sensical images).

2. Regarding VAEs, state whether each of the following statements is **true or false**, and explain:
    1. The latent-space distribution generated by the model for a specific input image is $\mathcal{N}(\vec{0},\vec{I})$.
    2. If we feed the same image to the encoder multiple times, then decode each result, we'll get the same reconstruction.
    3. Since the real VAE loss term is intractable, what we actually minimize instead is it's upper bound, in the hope that the bound is tight.
    
**Answer**

1. **False**. In VAE model we sample from a space depcited by $\mathcal{N}( \bb{\mu} _{\bb{\alpha}}(\bb{x}),  \mathrm{diag}\{ \bb{\sigma}^2_{\bb{\alpha}}(\bb{x}) \} )$ which is an approximation of $\mathcal{N}(\vec{0},\vec{I})$. The training process involves better approximating between the distributions.

2. **False**. We use the reparametrization trick which adds a random effect to the latent space. Since when feeding an image to the encoder we represent it in the latent space, each instance of the image would receive random effect and noise, therfore the output can be different.

3. **True**. The expectation in the loss formula is intractable, however we can obtain the ELBO (evidence lower bound) when deriving the optimization problem, and so we would like to minimize the negative value of it, which is what we did in HW4.

3. Regarding GANs, state whether each of the following statements is **true or false**, and explain:
    1. Ideally, we want the generator's loss to be low, and the discriminator's loss to be high so that it's fooled well by the generator.
    2. It's crucial to backpropagate into the generator when training the discriminator.
    3. To generate a new image, we can sample a latent-space vector from $\mathcal{N}(\vec{0},\vec{I})$.
    4. It can be beneficial for training the generator if the discriminator is trained for a few epochs first, so that it's output isn't arbitrary.
    5. If the generator is generating plausible images and the discriminator reaches a stable state where it has 50% accuracy (for both image types), training the generator more will further improve the generated images.
     
**Answer**
1. **False**: The discriminator's loss is comprised of two terms: one represnts its inability to distiguish generated images (which we do want it to be high) and the second is its ability to distiguish real images (which we want to be low). Therfore ideally we wouldn't want the loss of the descriminator to be alwas high as this might indicate that our model is not identifying the real images.
For that same manner we wouldn't want it to be completly zero, and rather keep it steady with lower variance (as read in the reference in HW4)
2. **False**: As we answered in part 3, When training the discriminator, we use the generator as a "black box" to generate some samples as adversarial inputs. We optimize  𝛿  for the discriminator and  𝛾  for the generator which are different parameters. We wouldn't want that when we optimize 𝛿, the generator will also be stepped as it creates an ill-defined optimization.
3. **True**: When we trained the generator, we trained it as if we sampled from a $\mathcal{N}(\vec{0},\vec{I})$ space. Therefore when generating a new image we sample from $\mathcal{N}(\vec{0},\vec{I})$.
4. **True**: It may be beneficial since if the discriminator is better trained (and does not output arbitrary classifications) it will probably force the generator to produce better quality images (as the discriminator will succeed more often)
5. **False**: We would assume that if the discriminator has a 50% accuracy it cannot distiguish anymore between real images and generated images and this can be explained by the fact that the generator fools the descriminator to a point where it thinks generating and real images are the same.
So assuming we would not try to improve anymore the descriminator, the generator will not improve as well, as the optimization tries to optimize them both step by step, and as their losses are a "zero-sum game" we would expect that the generator wold not improve.

### Graph Neural Networks

1. You have implemented a graph convolutional layer based on the following formula, for a graph with $N$ nodes:
$$
\mat{Y}=\varphi\left( \sum_{k=1}^{q} \mat{\Delta}^k \mat{X} \mat{\alpha}_k + \vec{b} \right).
$$
    1. Assuming $\mat{X}$ is the input feature matrix of shape $(N, M)$: what does $\mat{Y}$ contain in it's rows?
    1. Unfortunately, due to a bug in your calculation of the Laplacian matrix, you accidentally zeroed the row and column $i=j=5$ (assume more than 5 nodes in the graph).
What would be the effect of this bug on the output of your layer, $\mat{Y}$?


**Answer**:

1. 𝑌  is the out of the network. As X passed thorugh the network, each row of Y is the embedding vector of each node in the graph. 
Each layer of the graph is calculated by the previous layer features and specifically from nodes that are connected.
The context is created by the weights and bias of the neighbors 𝚫𝑘.

2. Removing row and column 5 effectively removes node N.5 from the graph, and all of its edges to other nodes. Therefore we eliminate any contibution that node N.5 (and its weights and bias) had towards its neighbors (and nodes afterwards that were affected by it)

2. We have discussed the notion of a Receptive Field in the context of a CNN. How would you define a similar concept in the context of a GCN (i.e. a model comprised of multiple graph convolutional layers)?

**Answer**
We can define the receptive field of a node in GCN as the number of nodes that influced its value thoughout the graph.
That includes all connected rings to the node (and their receptive fields) up to the q'th ring, as learned in tutorials.